In [1]:
''' initialise development environment '''

# set auto reload imported modules tagged
%load_ext autoreload
%autoreload 2


In [2]:
''' import optics package '''

# add custom python packages directory to path
import sys
sys.path.append('/home/brendan/dev/optics')


%matplotlib widget


# import path tracing and image transformation engine
import optics


In [3]:
''' Imports '''

# nd array manipulation
import numpy as np

# image manipulation
from scipy import ndimage

# plotting with matplotlib, interactive notebook, 3d toolkit

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# colour mapping helpers
import matplotlib.colors as colors
import matplotlib.cm as cmx


In [4]:
''' Test Sphere Scale and Axis '''


## plot ray paths

# initialise 3d figure
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(111, projection='3d')

ax.set_xlim(-2., 2.)
ax.set_ylim(-2., 2.)
ax.set_zlim(-2., 2.)


n2 = 1.0
rev = False
C = np.array([0., 0., 0.])
r = 1.

es = [1., 0.8]

cnorm = colors.Normalize(vmin = 0, vmax = len(es)); smap = cmx.ScalarMappable(norm = cnorm, cmap = 'magma_r')

for i in range(len(es)):
    
    c = smap.to_rgba(i)
    
    #e = np.array([ es[i] , 1., 1.])
    e = np.array([ 1., es[i], 1.])

    # get optic points in 3d for plotting
    x, y, z = optics.plot_3d_ellipsoid(C, r, e, rev, 0., full = False)
    # plot ellipsoid
    ax.plot_wireframe(x, y, z,  rstride = 3, cstride = 3, color = c, alpha = 0.5, label = '{}'.format(es[i]))


# format and display figure
plt.legend()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
''' Test Sphere Scale and Axis '''


## plot ray paths

# initialise 3d figure
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(111, projection='3d')

ax.set_xlim(-2., 2.)
ax.set_ylim(-2., 2.)
ax.set_zlim(-2., 2.)


n2 = 1.0
rev = False
C = np.array([0., 0., 0.])
r = 1.

es = [1., 1.2]

cnorm = colors.Normalize(vmin = 0, vmax = len(es)); smap = cmx.ScalarMappable(norm = cnorm, cmap = 'magma_r')

for ei in range(len(es)):
    
    c = smap.to_rgba(ei)
    
    #e = np.array([ es[i] , 1., 1.])
    e = np.array([ 1., es[ei], 1.])

    # generate and plot lens (ellipsoid)
    rx, ry, rz = np.array(e) * r

    # set of all spherical angles:
    #u = np.linspace(0.5 * np.pi, 1.5 * np.pi, 50)
    u = np.linspace(0.5 * np.pi, 1.5 * np.pi, 50)
    if True:
        k = np.linspace(0., 2*np.pi, 50)
    else:
        k = np.linspace(0., np.pi, 50)

    theta = np.deg2rad(30)
    rotate = np.array([
        [1,0,0],
        [0, np.cos(theta), -np.sin(theta)],
        [0, np.sin(theta), np.cos(theta)]])
        
    # Cartesian coordinates that correspond to the spherical angles:
    # (this is the equation of an ellipsoid):
    x = rx * np.outer(np.cos(u), np.sin(k))# + C[0]
    y = ry * np.outer(np.sin(u), np.sin(k))# + C[1]
    z = rz * np.outer(np.ones_like(u), np.cos(k))# + C[2]
    
    for i in range(len(x)):
        for j in range(len(x)):
            [x[i,j],y[i,j],z[i,j]] = np.dot([x[i,j],y[i,j],z[i,j]], rotate) + C
    
    # plot ellipsoid
    ax.plot_wireframe(x, y, z,  rstride = 3, cstride = 3, color = c, alpha = 0.5, label = '{}'.format(es[ei]))


# format and display figure
plt.legend()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
## test ray-optic refraction path calculation and 3d display

# initialise 3d figure
_w = 8; _h = 8
fig = plt.figure(figsize = (_w, _h))
#fig.canvas.layout.width = '{}in'.format(_w); fig.canvas.layout.height= '{}in'.format(_h)
ax = fig.add_subplot(111, projection='3d')

ax.set_xlim(-3., 3.)
ax.set_ylim(-3., 3.)
ax.set_zlim(-3., 3.)


# ellipse centre, radius, and axis vectors
C = np.array([2., 1., -1.])
r = 2.5
#e = [0.4, .6, .6]
e = np.array([0.6, 1.3, 1.])

rx, ry, rz = np.array(e) * r

# Set of all spherical angles:
u = np.linspace(0.5 * np.pi, 1.5 * np.pi, 50)
k = np.linspace(0., np.pi, 50)

# Cartesian coordinates that correspond to the spherical angles:
# (this is the equation of an ellipsoid):
x = rx * np.outer(np.cos(u), np.sin(k))# + C[0]
y = ry * np.outer(np.sin(u), np.sin(k))# + C[1]
z = rz * np.outer(np.ones_like(u), np.cos(k))# + C[2]


# pure rotation about primary axis, input angle in degrees
theta = 45.
_theta = np.deg2rad(theta)
rotate = np.array([[1,0,0], [0, np.cos(_theta), -np.sin(_theta)], [0, np.sin(_theta), np.cos(_theta)]])

for i in range(len(x)):
    for j in range(len(x)):
        [x[i,j],y[i,j],z[i,j]] = np.dot([x[i,j],y[i,j],z[i,j]], rotate) + C


# plot ellipsoid
ax.plot_wireframe(x, y, z,  rstride = 2, cstride = 2, color = 'k')


for Ly in np.arange(-1., 2., .5)[:]:

    # point on line (origin) and direction vector
    #L0 = [.0, 2., .5]
    L0 = [.0, Ly, .5]
    v = [1., .2, -.1]

    icepts = optics.engine.get_intercept(C, r, e, L0, v, theta)

    # define ray line points
    line = np.array([L0]).T + np.arange(-3., 5., 1.) * np.array([v]).T

    # plot ray line from points
    ax.plot( line[0,], line[1,], line[2,], color = 'r', alpha = 0.9)

    # plot intercept points
    for icept in icepts[-1:]:
        ax.plot(*icept, 'or')

    # point of intercept (on tangent plane)
    p = icepts[-1][:,0]

    # calculate surface normal vector
    n = optics.engine.get_surface_normal(C, e, p)

    # calculate d coefficient from point and normal vector
    d = p @ n - r

    # define ray line points
    line = np.array([p]).T + np.arange(p[0] - 2., p[0] + 1.) * np.array([n]).T
    # plot ray line from points
    ax.plot( line[0,], line[1,], line[2,], color = 'b', alpha = 0.9)


    # create y,z mesh
    yy, zz = np.meshgrid(np.arange(p[1] - 2., p[1] + 3.), np.arange(p[2] - 2., p[2] + 3.))
    # calculate corresponding x
    x = (r - (yy*n[1]) - (zz*n[2]) + d) / n[0]

    # plot ellipsoid
    ax.plot_wireframe(x, yy, zz,  rstride = 1, cstride = 1, color = 'g', alpha = 0.2)

    
    n1 = 1.0
    n2 = 1.5
    
    V = optics.engine.get_refracted_vector(n1, n2, n, v)

    # define ray line points
    line = np.array([p]).T + np.arange(p[0] - 2., p[0] + 1.) * np.array([V]).T
    # plot ray line from points
    ax.plot( line[0,], line[1,], line[2,], color = 'purple', alpha = 0.8)
    

# format and display figure
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
''' define test optic '''

cornea_f_dist = 0.
cornea_f_rad = 2.

cornea_pow = 1.1
cornea_sph = 0.

opts = [
    { # cornea
        'centre': np.array([cornea_f_dist, 0., 0.]),
        'opt_den': 1.377,
        'scale': np.array([cornea_pow, cornea_sph, 1.]),
        'radius': cornea_f_rad,
        'rev': False,
    },
]


In [19]:
''' generate test rays'''

rays = []

# calculate ray initial positions
rng = 1.
stp = 0.5

for Lz in np.arange(-rng, rng + stp, stp)[:]:
    for Ly in np.arange(-rng, rng + stp, stp)[:]:

        # origin ray position
        L0 = np.array([0., Ly, Lz])

        # normalised direction vector
        v = np.array([1., 0., 0.])
        v = v / np.linalg.norm(v)

        # store ray
        ray = np.concatenate([L0, v])
        rays.append(ray)

print(len(rays))

25


In [21]:
ray = rays[12]
ray

array([0., 0., 0., 1., 0., 0.])

In [28]:
L0 = ray[:3]
v = ray[3:]

print(L0, v)

# iterate over optics in path
optic = opts[0]

# get optic (ellipsoid): centre, radius, axes scale, refractive index, reverse flag
C = optic['centre']
r = optic['radius']
e = optic['scale']

print(C, r, e)

[0. 0. 0.] [1. 0. 0.]
[0. 0. 0.] 2.0 [1.1 0.  1. ]


In [29]:
# ellipse centre, radius, and axis vectors
C = np.array([C]).T

print('C\n', C)

# point on line (origin) and direction vector
L0 = np.array([L0]).T
v = np.array([v]).T

print('L0\n', L0)
print('v\n', v)


C
 [[0.]
 [0.]
 [0.]]
L0
 [[0.]
 [0.]
 [0.]]
v
 [[1.]
 [0.]
 [0.]]


In [30]:
# Translation Matrix
T = np.identity(4)
T[:-1,-1:] = C

print('T\n', T)

T
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [46]:

## generate rotation matrix from angle theta (deg.) around primary axis

theta = np.deg2rad(30)

rotate = np.array([
    [1,0,0],
    [0, np.cos(theta), -np.sin(theta)],
    [0, np.sin(theta), np.cos(theta)]
])

## fix rotation if required in future

# Rotation/Scale Matrix ?
R = np.identity(4)
R[:-1,:-1] = rotate

print('R\n', R)

#print(R)

R
 [[ 1.         0.         0.         0.       ]
 [ 0.         0.8660254 -0.5        0.       ]
 [ 0.         0.5        0.8660254  0.       ]
 [ 0.         0.         0.         1.       ]]


In [51]:
np.linalg.norm(rotate[:,2])

1.0

In [31]:
## fix rotation if required in future

# Rotation/Scale Matrix ?
R = np.identity(4)
R[:-1,:-1] = np.identity(3) * np.array(e).T

print('R\n', R)

R
 [[1.1 0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  1.  0. ]
 [0.  0.  0.  1. ]]


In [41]:

a = np.array([[1., 0., 0.]]).T# * e[0]
b = np.array([[0., 1., 0.]]).T# * e[1]
c = np.array([[0., 0., 1.]]).T# * e[2]

print('a\n', a)
print('b\n', b)
print('c\n', c)


# Scale/Rotate Matrix ?
S = np.identity(4)
S[0,0] = np.linalg.norm(a)
S[1,1] = np.linalg.norm(b)
S[2,2] = np.linalg.norm(c)

print('S\n', S)

a
 [[1.]
 [0.]
 [0.]]
b
 [[0. ]
 [1.2]
 [0. ]]
c
 [[0.]
 [0.]
 [1.]]
S
 [[1.  0.  0.  0. ]
 [0.  1.2 0.  0. ]
 [0.  0.  1.  0. ]
 [0.  0.  0.  1. ]]


In [ ]:
# Transformation Matrix
M = T @ R #@ S


# rescaled sphere centre
C_ = np.array([[0, 0, 0, 1]]).T

# transformed line origin
L0_ = np.linalg.inv(M) @ np.row_stack([L0,[1]])
# difference line vector
w = L0_ - C_

# transformed line vector
v_ = np.linalg.inv(M) @ np.row_stack([v,[0]])


# coefficients of quadratic intersection
a_ = (v_.T @ v_)[0]
b_ = 2*((v_.T @ w)[0])
c_ = (w.T @ w)[0] - r**2

# descriminant of quadratic
d_ = b_**2 - 4*a_*c_


# intersection points
if d_ > 0:
    t1 = (-b_ + np.sqrt(d_)) / (2*a_)
    t2 = (-b_ - np.sqrt(d_)) / (2*a_)
    L_int1 = L0 + t1 * v
    L_int2 = L0 + t2 * v
    icept = [L_int1, L_int2]


# tangent intercept
elif d_ == 0:
    t = -b_ / (2*a_)
    L_int = L0 + t * v
    icept = [L_int]


# no intercept
else:
    icept = []

